## Mood map data preparation

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('location_4Ws_bluetooth.csv')

In [ ]:
df = df.drop(["Unnamed: 0"], axis=1)
df

## Mood-map with activities

For each place (meaning for each unique combination of latitude and longitude), build a new column, containing a list of all the activities done in that place (not a set yet, as I need to know how many times each activity was done).
Then, another column, having the average mood for each place (ignoring NaN when doing the computation).
Finally, a column having as value the frequency of that location.

In [ ]:
df = df[df['mood'].notna()] 
df = df[df['what'].notna()] # deleted 2669 rows
unique_city = df.City.unique()
unique_city.sort()
#unique_city

In [ ]:
df.what.unique()

In [ ]:
wht = []
for a in range(0,len(df)):
    wht.append([])
df["what_lst"] = wht

In [ ]:
df = df.sort_values(by=['City'], ascending=True) 
df = df.reset_index()
df

In [ ]:
df = df.drop(["index"], axis=1)

In [ ]:
from statistics import mean

# activities list
index_city=0
index_row=0
what_list=[]
c=0
start=0
# avg mood 
mood_lst_per_loc = []
# place
pl = []

while True:
    i=unique_city[index_city]
    if df.iloc[index_row].at['City'] == i:
        what_list.append(df.iloc[index_row].at['what']) # list of activities
        mood_lst_per_loc.append(df.iloc[index_row].at['mood']) # avg mood
        pl.append(df.iloc[index_row].at['place']) # place
        index_row+=1
        c+=1
        if index_row == len(df.City):
            # add last 
            avgM = mean(mood_lst_per_loc)
            # build dictionary of places with their freq 
            d={}
            for el in set(pl):
                d[el]=pl.count(el)
            for r in range(start,len(df.what_lst)):
                df.at[r,'what_lst'] = what_list
                df.at[r,'avg_mood'] = avgM
                df.loc[r, 'places_dict'] = [d]
            #if len(set(pl))>1:
                #print(start,c)
            break
            
    if df.iloc[index_row-1].at['City'] != df.iloc[index_row].at['City']:
        avgM = mean(mood_lst_per_loc)
        d={}
        for el in set(pl):
            d[el]=pl.count(el)
        for r in range(start,c):
            df.at[r,'what_lst'] = what_list
            df.at[r,'avg_mood'] = avgM
            df.loc[r, 'places_dict'] = [d]
        #if len(set(pl))>1:
                #print(start,c)
        # reset       
        what_list=[]
        start=c
        index_city+=1
        pl=[]
        

In [ ]:
'''
A few geographic places have as places_lst a list with more than one element

This may be due to the fact that some timediaries were recorded some time after (and not in real time), 
thus the personal location given by the user is an approximation.
The GPS caught a location, but the user inputted that at that specific time they were in some places.

Most of all, this may also depend on the way data were processed: GPS location is collected every one minute, 
while time diaries are collected every half an hour.
According to the rule used for merging the two information, each single place coming from timediaries is matched with 30
data points coming from the GPS. (cfr. match_location_mood_bluetooth.ipynb for details)

Thus change: instead of a list use a dictionary with value corresponding to the counting.
'''
# example:
# print(df.iloc[1598].at['places_lst']) # ['Home apartment /room', 'In the street', 'Classroom / Study hall', 'Grocery Shop', 'Another outdoor place']
print(df.iloc[1598].at['places_dict'])

In [ ]:
# To verify/have an idea of what kind of place correspond to the GPS location for real, 
# let’s count how many time the place was inputted for each GPS location; 
# the most frequent should be the more likely to be correct.
# That's why the column for places, actually correspond to a dictionary having as keys the places
# and as values the number of time that place appeared for the specific GPS location considered.

In [ ]:
#df['City'].value_counts() 
df['frequency'] = df['City'].map(df['City'].value_counts(normalize=False)) # not normalized as numbers becomes very small if normalized wrt the whole datasets (order 10^-4) 

In [ ]:
df

In [ ]:
df = df.drop(["mood", "what", "place", "bluetooth", "with_who", "timestamp"], axis=1) # for the mood map task we do not need these columns
df

In [ ]:
# remove rows having same city (the information of all rows have been stored in the other columns)
df = df.drop_duplicates(subset=['City'])
df

In [ ]:
df.to_csv('mood_map.csv')

## Bluetooth (with who?) map data preparation

Re-load dataset. Keep the column "bluetooth" and "with_who", as they will be used to answer questions such as "with who you are in that place?", "which activities you do with who?", "how the company influence your mood?", and so on.
Remove only rows where both "bluetooth" and "with_who" are NaN.
Create a new column, where each cell contains a list of the hours that location was visited.
Create a new column with a dictionary containing as keys the day of the week and as values the number of time that place is present in the dataframe for that day of the week.
In this way, it will be possible to take into consideration of there are some places that are visited in specific hours or specific time of the day.

Create a new column where, for each unique place, there will be a dictionary of the "with_who" (coming from timediaries) options present for that place, with as value the number of time that company was inputted in that place.

Keep these new columns, as well as lat, long, city, what, mood, with_who and bluetooth.

In [ ]:
import pandas as pd
df = pd.read_csv('location_4Ws_bluetooth.csv')
df = df.drop(["Unnamed: 0"], axis=1)
df

In [ ]:
df = df.sort_values(by=['City'], ascending=True) 

In [ ]:
# drop row if both "with_who" and "bluetooth" values are NaN
df = df.dropna(subset=["with_who", "bluetooth"], how='all')
df

In [ ]:
df = df.reset_index()

In [ ]:
df = df.drop(["index"], axis=1)
df

In [ ]:
unique_city = df.City.unique()

index_city=0
index_row=0
c=0
start=0
# place
pl = []
# time of day 
t = []
# who
who = []


while True:
    i=unique_city[index_city]
    if df.iloc[index_row].at['City'] == i:
        t.append(str(df.iloc[index_row].at['timestamp'])[-4:]) # time of the day
        if pd.isna(df.iloc[index_row].at['place'])==False:
            pl.append(df.iloc[index_row].at['place']) # place
        if pd.isna(df.iloc[index_row].at['with_who'])==False:
            who.append(df.iloc[index_row].at['with_who'])
        index_row+=1
        c+=1
        if index_row == len(df.City):
            # place
            d={}
            for el in set(pl):
                d[el]=pl.count(el)
            # with who
            dw={}
            for elw in set(who):
                dw[elw]=who.count(elw)
            for r in range(start,len(df.City)):
                df.loc[r, 'places_dict'] = [d]
                df.at[r, 'time_of_day'] = t  
                df.loc[r, 'who_lst'] = [dw] 
            break
            
    if df.iloc[index_row-1].at['City'] != df.iloc[index_row].at['City']:
        # place
        d={}
        for el in set(pl):
            d[el]=pl.count(el)
        # with who
            dw={}
            for elw in set(who):
                dw[elw]=who.count(elw)
        for r in range(start,c):
            df.loc[r, 'places_dict'] = [d]
            df.at[r, 'time_of_day'] = t
            df.loc[r, 'who_lst'] = [dw] 
        
        # reset       
        pl=[]
        start=c
        index_city+=1
        t=[]
        who=[]

In [ ]:
# test:
# from datetime import date
# import calendar
import pandas as pd

temp = pd.Timestamp('2020-11-25')
print(temp.day_name())

In [ ]:
# day of the week

day = []

unique_city = df.City.unique()

index_city=0
index_row=0
c=0
start=0

while True:
    i=unique_city[index_city]
    if df.iloc[index_row].at['City'] == i:
        str_day = str(df.iloc[index_row].at['timestamp'])[:-4]
        temp = pd.Timestamp(str_day[:4]+'-'+str_day[4:6]+'-'+str_day[6:])
        day.append(temp.day_name()) # day of the week
        
        
        index_row+=1
        c+=1
        if index_row == len(df.City):
            d={}
            for el in set(day):
                d[el]=day.count(el)
            for r in range(start,len(df.City)):
                df.loc[r, 'day'] = [d]              
            break
            
    if df.iloc[index_row-1].at['City'] != df.iloc[index_row].at['City']:
        d={}
        for el in set(day):
            d[el]=day.count(el)
        for r in range(start,c):
            df.loc[r, 'day'] = [d]
        
        # reset       
        day=[]
        start=c
        index_city+=1
        

In [ ]:
df = df.drop(["place"], axis=1)

In [ ]:
df

In [ ]:
df = df.drop_duplicates(subset=['lat', 'long', 'City', 'mood', 'bluetooth', 'with_who', 'what'])
df = df.drop(["timestamp"], axis=1) 
df = df.reset_index()

In [ ]:
df = df.drop(["index"], axis=1)

In [ ]:
df

In [ ]:
# save
df.to_csv('with_who_map.csv')